In [79]:
import random

class Player:
    """
    The bid method places a random bid between 0 and the player's available cash,
    up to the card's value. This method does not offer any cards from the player's
    portfolio in the bid, but you can modify it to include cards based on your preferred strategy.
    
    The accept_bid method selects the highest cash bid.
    If there are no bids with cash, it returns None. You can modify this method to
    consider bids with cards or implement other acceptance strategies.

    The sell_crypto method sells the card with the highest value in the player's portfolio.
    This method does not implement any selling restrictions, such as only allowing players
    to sell once during the game. You can add these restrictions or other selling strategies as needed.
    """
    def __init__(self, name, starting_cash, first_card, min_sale, buy_rate, bid_strat, accept_strat):
        # Define methods for bidding, accepting bids, and selling cryptocurrencies
        self.name = name
        self.cash = starting_cash
        self.portfolio = [first_card]
        self.bid_strat = bid_strat
        self.accept_strat = accept_strat
        self.min_sale = min_sale
        self.buy_rate = buy_rate

    def bid(self, card):
        # Return a tuple with the cash amount and a list of cards offered in the bid
        return self.bid_strat(card, self.cash, self.portfolio)

    def accept_bid(self, bids, card):
        # Return the chosen bid
        return self.accept_strat(bids, card, self.buy_rate)

    def sell_crypto(self):
        # Update the player's cash and portfolio if there is anything to sell:
        card_to_sell = max(self.portfolio, key=lambda card: card.value)
        sold = False

        while card_to_sell and len(self.portfolio) > 3:
            if card_to_sell.value > self.min_sale:
                self.cash += card_to_sell.value
                self.portfolio.remove(card_to_sell)
                card_to_sell = max(self.portfolio, key=lambda card: card.value)
                sold = True
            else:
                card_to_sell = None
        
        return sold

    def __str__(self):
        return f"{self.name} - Cash: ${self.cash}, Portfolio Value: ${sum(card.value for card in self.portfolio)}"



In [80]:
from collections import Counter
from random import random

def nearest_10(cash, max_pct):
    return round(random() * cash * max_pct / 10) * 10

def simple_bid(card, my_cash, my_cards):
    my_card_names = [my_card.name for my_card in my_cards]
    counted = Counter(my_card_names)
    eligible_cards = sorted([my_card for my_card in my_cards if my_card.name != card.name], key=lambda i: i.value)
    
    # Cap amount of cash up for grabs
    if my_cash > 150:
        eligible_cash = 150
    else:
        eligible_cash = my_cash

    cards_offered = []
    cash_offer = 0

    # If already owning multiple cards of this card, go hard -> bidding multiple other cards + up to 50% cash
    if card.name in my_card_names and card.name == counted.most_common(1)[0][0]:
        # Bid your top X cards, were X is the number of cards already owned!
        x = counted.most_common(1)[0][1]
        for i in range(x):
            if eligible_cards:
                cards_offered.append(eligible_cards.pop())
        cash_offer = nearest_10(eligible_cash, 0.5)

    else:
        # If card's name exists in my_cards, up to 30% cash, and highest other card
        if card.name in my_card_names:
            if eligible_cards:
                cards_offered.append(eligible_cards[0])
            cash_offer = nearest_10(eligible_cash, 0.5)

        # If card is new, and we have more than 2 cards, bid with second highest card:
        elif len(eligible_cards) > 2:
            cards_offered.append(eligible_cards[1])
        
        # If we don't have many cards, simply offer up to 40% cash
        else:
            cash_offer = nearest_10(eligible_cash, 0.4)
        
    return (cash_offer, cards_offered)

def simple_accept(bids, card, buy_rate):
    # If multiple cards in an offer, take that offer:
    if len(max(bids, key=lambda bid: len(bid[1]))[1]) > 1:
        highest_bid = max(bids, key=lambda bid: len(bid[1][1]))
    # If all offers have single cards, take highest cash offer: 
    else:
        highest_bid = max(bids, key=lambda bid: bid[1][0])
    
    # determine if highest bid is greater than the buy_rate
    cards_value = sum(card.value for card in highest_bid[1][1])
    if (highest_bid[1][0] + cards_value) <= buy_rate:
        return None
    
    return highest_bid


In [81]:
from random import random, shuffle

# Classes
class Card:
    # Initialize card with a reference to the crypto it belongs to and its value
    def __init__(self, name, value):
        self.name = name
        self.value = value

    def __str__(self):
        return f"{self.name} - ${self.value}"
    

class Game:
    # Initialize the game with the number of players and cryptocurrencies
    # Define methods for simulating a round, checking for a winner, and running the game

    def __init__(self, num_players, starting_cash, cards_per_crypto, winning_cash):
        self.deck, self.all_cards = create_deck(num_players + 1, cards_per_crypto)
        self.players = [
            Player(
                f"Player {i + 1}",
                starting_cash,
                Card(f"Card {i + 1}", i*10),
                200 + round(random() * 50 / 10) * 10, # min_sale
                10 + round(random() * 20 / 10) * 10, # buy_rate
                simple_bid,
                simple_accept,
            ) for i in range(num_players)
        ]
        for player in self.players:
            self.all_cards.append(player.portfolio[0])
        self.winning_cash = winning_cash

    def update_values(self, name, new_value):
        for card in self.all_cards:
            if card.name == name:
                card.value = new_value

    def play_round(self):
        if not self.deck:
            return False

        card = self.deck.pop()
        auctioneer = self.players.pop(0)
        
        # get bids from all other players:
        bids = [(player, player.bid(card)) for player in self.players]

        # auctioneer chooses a bid
        chosen_bid = auctioneer.accept_bid(bids, card)
        
        # put actioneer back in player stack, at the end
        self.players.append(auctioneer)
        
        if chosen_bid:
            bidder, (cash_amount, cards_offered) = chosen_bid
            bidder.cash -= cash_amount
            auctioneer.cash += cash_amount
            
            cards_value = sum(card.value for card in cards_offered)

            for card_offered in cards_offered:
                bidder.portfolio.remove(card_offered)
                auctioneer.portfolio.append(card_offered)

            bidder.portfolio.append(card)
            self.update_values(card.name, cash_amount + cards_value)
        else:
            auctioneer.cash -= auctioneer.buy_rate
            auctioneer.portfolio.append(card)
            self.update_values(card.name, auctioneer.buy_rate)

        # all players see if they'd like to sell:
        for player in self.players:
            player.sell_crypto()
            
        return True

    def check_winners(self):
        winners = []
        for player in self.players:
            if player.cash >= self.winning_cash:
                winners.append(player)
        return winners
    

# Functions
def create_deck(num_cryptos, cards_per_crypto):
    # Create a shuffled deck of cards with the specified number of cryptocurrencies
    deck = []
    for i in range(cards_per_crypto):
        for j in range(num_cryptos):
            deck.append(Card(f"Card {j + 1}", j*10))
    
    shuffle(deck)
    shuffle(deck)

    return (deck, [card for card in deck])


def play_game(game):
    winners = []
    while game.deck and not winners:
        # play a round:
        game.play_round()

        # look for a winner:
        if game.check_winners():
            winners = game.check_winners()
        
    print(f"Game complete!")
    print("Final standings:")
    for player in game.players:
        print(player)
    
    return game


In [82]:
game = Game(
    num_players = 5,
    starting_cash = 100,
    cards_per_crypto = 6,
    winning_cash = 1000
)

In [112]:
game.play_round()
for player in game.players:
    print(f"{player.name}, portfolio: {[(card.name, card.value) for card in player.portfolio]}, cash: {player.cash}")
print(f"cards left: {len(game.deck)}")

Player 1, portfolio: [('Card 6', 90), ('Card 2', 90), ('Card 4', 110)], cash: 780
Player 2, portfolio: [('Card 2', 90), ('Card 6', 90), ('Card 3', 270)], cash: 780
Player 3, portfolio: [('Card 6', 90), ('Card 5', 110), ('Card 2', 90), ('Card 4', 110), ('Card 5', 110)], cash: 1310
Player 4, portfolio: [('Card 1', 300), ('Card 1', 300), ('Card 1', 300)], cash: 350
Player 5, portfolio: [('Card 5', 110), ('Card 5', 110), ('Card 2', 90), ('Card 5', 110), ('Card 5', 110), ('Card 6', 90), ('Card 4', 110), ('Card 2', 90), ('Card 2', 90)], cash: 320
cards left: 6


In [68]:
game.players[1].name
game.players[1].min_sale
game.players[1].sell_crypto()

In [69]:
print([(card.name, card.value) for card in game.players[1].portfolio])

[('Card 1', 150), ('Card 3', 20), ('Card 1', 150), ('Card 2', 120), ('Card 1', 150), ('Card 5', 350), ('Card 5', 350), ('Card 5', 350)]


In [75]:
# Update the player's cash and portfolio if there is anything to sell:
portfolio = game.players[1].portfolio
card_to_sell = max(portfolio, key=lambda card: card.value)
sold = False

In [76]:
min_sale = 250
cash = 0